## Homework

> **Note**: it's very likely that in this homework your answers won't match 
> the options exactly. That's okay and expected. Select the option that's
> closest to your solution.

### Dataset

In this homework, we'll build a model for classifying various hair types. 
For this, we will use the Hair Type dataset that was obtained from 
[Kaggle](https://www.kaggle.com/datasets/kavyasreeb/hair-type-dataset) 
and slightly rebuilt. 

You can download the target dataset for this homework from 
[here](https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip):

```bash
wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
unzip data.zip
```

In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch. 

> **Note:** you will need an environment with a GPU for this homework. We recommend to use [Saturn Cloud](https://bit.ly/saturn-mlzoomcamp). 
> You can also use a computer without a GPU (e.g. your laptop), but it will be slower.


### Data Preparation

The dataset contains around 1000 images of hairs in the separate folders 
for training and test sets. 

### Reproducibility

Reproducibility in deep learning is a multifaceted challenge that requires attention 
to both software and hardware details. In some cases, we can't guarantee exactly 
the same results during the same experiment runs. Therefore, in this homework we suggest to:
* install tensorflow version 2.17.1
* set the seed generators by:

```python
import numpy as np
import tensorflow as tf

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
```

### Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(200, 200, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

For clarification about kernel size and max pooling, check [Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).



In [53]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError, CategoricalCrossentropy, CosineSimilarity
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__ 

'2.17.1'

In [8]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [13]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [21]:
train_ds = train_gen.flow_from_directory("../DATASETS/data/train/", target_size=(200, 200))

Found 800 images belonging to 2 classes.


In [22]:
train_ds.class_indices

{'curly': 0, 'straight': 1}

In [23]:
X, y = next(train_ds)
X.shape

(32, 200, 200, 3)

In [ ]:
base_model = Xception(weights="imagenet", include_top=False, input_shape=(200, 200, 3))
base_model.trainable = False

In [44]:
input_size = 200
learning_rate = 0.002
momentum = 0.8

model = Sequential([
    Input(shape=(input_size, input_size , 3)),
    Conv2D(filters=32, kernel_size=(3, 3), activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64, activation="relu"),
    Dense(1, activation="sigmoid")
])


optimizer = SGD(learning_rate=learning_rate, momentum=momentum)
loss = BinaryCrossentropy()

model.compile(
    optimizer=optimizer,
    loss=loss, 
    metrics=["accuracy"]
)


### Question 1

Since we have a binary classification problem, what is the best loss function for us?

* `mean squared error`
* `binary crossentropy` <-
* `categorical crossentropy`
* `cosine similarity`

> **Note:** since we specify an activation for the output layer, we don't need to set `from_logits=True`

> We have only 2 clases, so the best choose would be **binary crossentropy

### Question 2

What's the total number of parameters of the model? You can use the `summary` method for that. 

* 896 
* 11214912
* 15896912
* 20072512 <-

In [45]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

In [46]:
20072512 + 896 + 65

20073473

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/test directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and test sets. 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
```

In [47]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [48]:
train_generator = train_datagen.flow_from_directory(
    '../DATASETS/data/train/',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True)

Found 800 images belonging to 2 classes.


In [51]:
test_generator = val_datagen.flow_from_directory(
    '../DATASETS/data/test/',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True)

Found 201 images belonging to 2 classes.


In [50]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 200, 200, 3)
labels batch shape: (20,)


In [56]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 133s 3s/step - accuracy: 0.7833 - loss: 0.4515 - val_accuracy: 0.6418 - val_loss: 0.6527
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.7700 - loss: 0.4704 - val_accuracy: 0.6716 - val_loss: 0.5949
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 320ms/step - accuracy: 0.7851 - loss: 0.4665 - val_accuracy: 0.6915 - val_loss: 0.5886
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 312ms/step - accuracy: 0.8325 - loss: 0.3950 - val_accuracy: 0.6716 - val_loss: 0.6247
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 622ms/step - accuracy: 0.7994 - loss: 0.4499 - val_accuracy: 0.6269 - val_loss: 0.7090
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 70s 2s/step - accuracy: 0.8575 - loss: 0.3794 - val_accuracy: 0.6617 - val_loss: 0.6128
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 90s 2s/step - accuracy: 0.8503 - loss: 0.3783 - val_accuracy: 0.7065 - val_loss: 0.6124
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.8751 - loss: 0.3461 - val_accuracy: 0.6766 -

### Question 3

What is the median of training accuracy for all the epochs for this model?

* 0.10
* 0.32
* 0.50
* 0.72 <-


In [58]:
np.median(history.history["accuracy"]).round(2)

0.83

### Question 4

What is the standard deviation of training loss for all the epochs for this model?

* 0.028
* 0.068 <-
* 0.128
* 0.168


In [61]:
np.std(history.history["loss"]).round(3)

0.062

### Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=50,`
* `width_shift_range=0.1,`
* `height_shift_range=0.1,`
* `zoom_range=0.1,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`

In [62]:
train_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_generator.flow_from_directory(
    "../DATASETS/data/train/", 
    target_size=(input_size, input_size),
    class_mode="binary",
    batch_size=20,
    shuffle=True,
)

Found 800 images belonging to 2 classes.


### Question 5 

Let's train our model for 10 more epochs using the same code as previously.
> **Note:** make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.26
* 0.56 <-
* 0.86
* 1.16

In [63]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.6553 - loss: 0.7009 - val_accuracy: 0.6965 - val_loss: 0.5879
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 681ms/step - accuracy: 0.7029 - loss: 0.5956 - val_accuracy: 0.7164 - val_loss: 0.5498
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 20s 498ms/step - accuracy: 0.6834 - loss: 0.6070 - val_accuracy: 0.7065 - val_loss: 0.5692
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 29s 726ms/step - accuracy: 0.7107 - loss: 0.5736 - val_accuracy: 0.7114 - val_loss: 0.5683
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 20s 492ms/step - accuracy: 0.6400 - loss: 0.6076 - val_accuracy: 0.7065 - val_loss: 0.5416
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 20s 493ms/step - accuracy: 0.7367 - loss: 0.5481 - val_accuracy: 0.6766 - val_loss: 0.6260
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 20s 490ms/step - accuracy: 0.6978 - loss: 0.5600 - val_accuracy: 0.6965 - val_loss: 0.5845
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 20s 502ms/step - accuracy: 0.7349 - loss: 0.5446 - val_accurac

In [64]:
np.mean(history.history["val_loss"]).round(2)

0.57

### Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

* 0.31
* 0.51
* 0.71 <-
* 0.91

In [66]:
np.mean(history.history["val_accuracy"][-5:]).round(3)

0.7